In [1]:
import numpy as np
import sympy as sm
import scipy as sp

In [2]:
elementsData = dict()
elementsChars = ['H', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'mu', 'sigma', 'epsil']
elements = ['AlCl', 'AlCl2', 'AlCl3', 'GaCl', 'GaCl2', 'GaCl3', 'NH3', 'H2', 'HCl', 'N2', 'Al', 'Ga', 'AlN', 'GaN']

dataFile = open('data.txt', 'r')
lines = dataFile.readlines()
for line in lines:
    dataArr = line.rstrip().split(';')
    elementsData[dataArr[0]] = dict()
    for i in range(0, len(dataArr) - 1):
        elementsData[dataArr[0]][elementsChars[i]] = float(dataArr[i + 1])

In [3]:
R = 8.314  # Дж/(кмоль⋅К) - универсальная газовая постоянная
P_a = 100000  # Па – полное давление

In [4]:
'''
d - коэффициент диффузии (м2/сек)
compIndex - название элемента
temperature - температура
pressure - полное давление в системе (P_i_g - P_i_e)
'''
def d(compIndex, temperature, pressure):
    numerator = 2.628 * 10 ** (-2) * temperature ** (3 / 2)

    dataForIndex = elementsData[compIndex]
    dataForN2 = elementsData['N2']

    sigma = (dataForIndex['sigma'] + dataForN2['sigma']) / 2
    epsil = (dataForIndex['epsil'] * dataForN2['epsil']) ** (1 / 2)
    mu = (2 * dataForIndex['mu'] * dataForN2['mu']) / (dataForIndex['mu'] + dataForN2['mu'])
    omega = 1.074 * (temperature / epsil) ** (-0.1604)

    return numerator / (pressure * sigma * omega * mu ** (1 / 2))

"""
phi нужно для расчета энергии Гиббса (G)
compIndex - название элемента
temperature - температура
"""
def phi(compIndex, temperature):
    dataForIndex = elementsData[compIndex]

    x = temperature / 10 ** 4
    return dataForIndex['f1'] + \
           dataForIndex['f2'] * np.log(x) + \
           dataForIndex['f3'] / x ** 2 + \
           dataForIndex['f4'] / x + \
           dataForIndex['f5'] * x + \
           dataForIndex['f6'] * x ** 2 + \
           dataForIndex['f7'] * x ** 3
"""
g - Энергия Гиббса
compIndex - название элемента
temperature - температура
"""
def g(compIndex, temperature):
    dataForIndex = elementsData[compIndex]
    return dataForIndex['H'] - phi(compIndex, temperature) * temperature


In [5]:
#ета надо будет для первого задания
def k1(temperature):
    g1Diff = 2 * g('Al', temperature) + 2 * g('HCl', temperature) - 2 * g('AlCl', temperature) - g('H2', temperature)
    return np.exp(-g1Diff / (R * temperature)) / P_a

def k2(temperature):
    g2Diff = g('Al', temperature) + 2 * g('HCl', temperature) - g('AlCl2', temperature) - g('H2', temperature)
    return np.exp(-g2Diff / (R * temperature))

def k3(temperature):
    g3Diff = g('Al', temperature) - 2 * g('AlCl3', temperature) - 3 * g('H2', temperature) + 6 * g('HCl', temperature)
    return np.exp(-g3Diff / (R * temperature)) * P_a

#ета надо будет для второго задания
def k4(temperature):
    g4Diff = g('Ga', temperature) + 2 * g('HCl', temperature) - 2 * g('GaCl', temperature) - g('H2', temperature)
    return np.exp(-g4Diff / (R * temperature)) / P_a

def k5(temperature):
    g5Diff = g('GaCl2', temperature) + g('H2', temperature) - 2 * g('HCl', temperature) - g('Ga', temperature)
    return np.exp(g5Diff / (R * temperature))

def k6(temperature):
    g6Diff = 2 * g('GaCl3', temperature) + 3 * g('H2', temperature) - 6 * g('HCl', temperature) - 2 * g('Ga', temperature)
    return np.exp(g6Diff / (R * temperature)) * P_a

#ета надо будет для третьего задания
def k9(temperature):
    g9Diff = g('AlCl3', temperature) + g('NH3', temperature) - g('AlN', temperature) - 3 * g('HCl', temperature)
    return np.exp(-g9Diff / (R * temperature)) / P_a

def k10(temperature):
    g10Diff = g('GaCl', temperature) + g('NH3', temperature) - g('HCl', temperature) - g('H2', temperature) - g('GaN', temperature)
    return np.exp(-g10Diff / (R * temperature))

In [6]:
def newton_method(system, variables, eps=1e-9):
    jacobian = sm.lambdify(variables, system.jacobian(variables))
    system = sm.lambdify(variables, system)
    
    x = np.random.uniform(0.1, 1., len(variables)).tolist()
    delta = 1
    steps = 0
    while np.linalg.norm(delta) > eps:
        delta = sp.linalg.lu_solve(sp.linalg.lu_factor(jacobian(*x)), -system(*x))
        x = delta.flatten() + x
        steps += 1
    rsa = np.abs(system(*x.tolist())).sum()
    return x, rsa, steps

# Задание №1

In [7]:
"""
Парциальные давления компонент вне диффузионного пограничного слоя (Па)
PS Даны в условии поэтому дефайним только для этого задания
ind - 'AlCl', 'GaN, ... 
"""
# def first_calcPg(ind):
#     if ind == 'AlCl' or ind == 'AlCl2' or ind == 'AlCl3' or ind == 'H2':
#         return 0
#     if ind == 'HCl':
#         return 10000
#     if ind == 'N2':
#         return 90000
#     raise Exception('Unexpected substance: ' + ind)


def system1(pg_HCl=10000.0, p=100000, T=(350.0 + 273.0)):
    p_AlCl, p_AlCl2, p_AlCl3, p_HCl, p_H2 = sm.symbols('p_AlCl, p_AlCl2, p_AlCl3, p_HCl, p_H2')
    d_HCl = d('HCl', T, p)
    d_H2 = d('H2', T, p)
    d_AlCl = d('AlCl', T, p)
    d_AlCl2 = d('AlCl2', T, p)
    d_AlCl3 = d('AlCl3', T, p)
    equations = []
    equations.append(p_HCl ** 2 - k1(T) * p_AlCl ** 2 * p_H2)
    equations.append(p_HCl ** 2 - k2(T) * p_AlCl2 * p_H2)
    equations.append(p_HCl ** 6 - k3(T) * p_AlCl3 ** 2 * p_H2 ** 3)
    equations.append(d_HCl * (p_HCl - pg_HCl) + 2 * d_H2 * p_H2)
    equations.append(d_AlCl  * p_AlCl  + 
                 2 * d_AlCl2 * p_AlCl2 + 
                 3 * d_AlCl3 * p_AlCl3 + 
                     d_HCl * (p_HCl - pg_HCl))
    system = sm.Matrix(equations)
    return system, (p_AlCl, p_AlCl2, p_AlCl3, p_HCl, p_H2)

print('One of solutions:')
system, variables = system1(T=350 + 273)
dict(zip(variables, newton_method(system, variables)[0]))

One of solutions:


{p_AlCl: 0.004220558330250097,
 p_AlCl2: 0.0001862660837734311,
 p_AlCl3: 5983.089564621574,
 p_HCl: 0.0001446675551736787,
 p_H2: 1561.368796088631}